# OMOP to Patient

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

curOMOP = conn.cursor()

In [ ]:
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### Person to Patient

In [1]:
query= """
    SELECT p.person_id , p.gender_concept_id, p.race_concept_id, p.year_of_birth, p.location_id 
    FROM person p;
"""

insert_query = """
    INSERT INTO patient (id, sex, race, birthYear, countryOfResidence)
    VALUES (%s, %s, %s, %s, %s);
"""
curOMOP.execute(query)

results = curOMOP.fetchall()

for row in results:
    id_person = row[0]
    sex = row[1]
    race = row[2]
    birthYear = row[3]
    countryOfResidence = row[4]
    curIDEA.execute(insert_query, (id_person, sex, race, birthYear,countryOfResidence))
    conn2.commit()
    

SyntaxError: incomplete input (2615831892.py, line 1)

### Observation to Patient

In [2]:
query= """
    SELECT id
    FROM patient;
"""


observation_vocab_values_concept_id = { 
    1585856: 'smoking',
    903650: 'cigarettesPackYearsSmokedDuringLife',
    1586197: 'alcohol',
    46235351: 'comorbidity',
    4329847: 'myocardialInfarction',
    319835: 'congestiveHeartFailure',
    321052: 'peripheralVascularDisease',
    381316: 'cerebrovascularAccident',
    4182210: 'dementia',
    4186898: 'chronicPulmonaryDisease',
    4344165: 'connectiveTissueDisease',
    4177703: 'ulcer',
    194984: 'mildLiverDisease
    201820: 'diabetes',
    16020753: 'diabetesWithEndOrganDamage',
    374022: 'hemiplegia',
    198124: 'moderateToSevereRenalDisease',
    1621181:'metastaticSolidTumor',
    443392: 'solidTumor',
    317510: 'leukemia',
    44499278: 'lymphoma',
    437233: 'multipleMyeloma',
    4267414: 'aids',
    36305384: 'easternCooperativeOncologyGroupPerformanceStatusAtDiagnosis',
    36303470: 'ecogPsLabel',
    4169154: 'karnofsyIndexAtDiagnosis',
    36303744: 'karnofsyIndexLabel',
    37204336: 'noGeneticSyndromeWho2020'
}

areNumbers = [
    "easternCooperativeOncologyGroupPerformanceStatusAtDiagnosis",
    "karnofsyIndexAtDiagnosis"
]

curOMOP.execute(query)

results = curIDEA.fetchall()

queryForCodes="""
    SELECT value_as_concept_id FROM observation WHERE observation.observation_concept=%s AND observation.person_id=%s;
"""
queryForNumbs="""
    SELECT value_as_number FROM observation WHERE observation.observation_concept=%s AND observation.person_id=%s;
"""

queryoccurrenceOfOtherCancer="""
    SELECT value_as_concept_id FROM observation WHERE observation.observation_concept=1340204 AND observation.person_id=%s
    AND value_as_concept_id IN (273629,4170755,4121697,42535584,4295112);
"""
querypreviousCancerTreatment="""
    SELECT value_as_concept_id FROM observation WHERE observation.observation_concept=1340204 AND observation.person_id=%s
    AND value_as_concept_id=4266186;
"""

for row in results:
    for code in observation_vocab_values_concept_id.keys():
        column_name=observation_vocab_values_concept_id.get(code)    
        update = f"""
            UPDATE patient
            SET {column_name} = %s
            WHERE id = %s;
        """
            curOMOP.execute(queryForCodes,(code,row))
            value = curIDEA.fetchone()
        else:
            curOMOP.execute(queryForNumbs,(code,row))
            value = curIDEA.fetchone()
        
        curIDEA.execute(update, (value,row))
        conn2.commit()
    column_name='occurrenceOfOtherCancer'
    update = f"""
            UPDATE patient
            SET {column_name} = %s
            WHERE id = %s;
        """
    curOMOP.execute(queryoccurrenceOfOtherCancer,(row))
    value = curIDEA.fetchone()
    curIDEA.execute(update, (value,row))
    conn2.commit()
    column_name='previousCancerTreatment'
    update = f"""
            UPDATE patient
            SET {column_name} = %s
            WHERE id = %s;
        """
    curOMOP.execute(querypreviousCancerTreatment,(row))
    value = curIDEA.fetchone()
    curIDEA.execute(update, (value,row))
    conn2.commit()

SyntaxError: unterminated string literal (detected at line 24) (4103621420.py, line 24)